# Bigram
https://towardsdatascience.com/text-analysis-basics-in-python-443282942ec5

# Loading data

In [ ]:
import pandas as pd 
import numpy as np

#agar mudah, letakkan file data dalam satu folder dengan file jupiter notebook nya

filedata = 'discussion'

dataSB = pd.read_excel(filedata+".xlsx", sheet_name="Sheet1")

dataSB.head()

# lower case

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
dataSB['textdata'] = dataSB['message'].str.lower()

print('Case Folding Result : \n')
print(dataSB['textdata'].head(5))

In [ ]:
#jika AttributeError: 'float' object has no attribute 'replace'

#import ast
dataSB['textdata'] = dataSB['textdata'].astype(str)

# Tokenizing
Memecah teks menjadi perkata, dan membersihkan simbol-simbol

In [ ]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
dataSB['textdata'] = dataSB['textdata'].apply(remove_tweet_special)

In [ ]:
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataSB['textdata'] = dataSB['textdata'].apply(remove_number)


In [ ]:
#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataSB['textdata'] = dataSB['textdata'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataSB['textdata'] = dataSB['textdata'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataSB['textdata'] = dataSB['textdata'].apply(remove_singl_char)

#dataSB['textdata']

In [ ]:
def remove_special_bigram(text):
    text = text.replace("kolom komentar", " ").replace("jutaan rupiah", " ").replace("rp", " ")
    
    text = text.replace("apple amazon", " ")
    text = text.replace("pendiri apple", " ")
    
    return text 
                
dataSB['textdataClean'] = dataSB['textdata'].apply(remove_special_bigram)

In [ ]:
def remove_special_bigram2(text):
    
    text = text.replace("selamat malam", " ")
    text = text.replace("konsep prinsip", " ")
    text = text.replace("terima kasih", " ")
    text = text.replace("kesamaan konsep", " ")
    text = text.replace("dimiliki amazon", " ")
    text = text.replace("apple amazon", " ")
    text = text.replace("prinsip dimiliki", " ")

    return text 
                
dataSB['textdataClean'] = dataSB['textdataClean'].apply(remove_special_bigram2)

# Proses

In [ ]:
from textblob import TextBlob
dataSB['polarity'] = dataSB['textdataClean'].apply(lambda x: TextBlob(x).polarity)
dataSB['subjective'] = dataSB['textdataClean'].apply(lambda x: TextBlob(x).subjectivity)
dataSB[0:5]

In [ ]:
from nltk.corpus import stopwords
stoplist = stopwords.words('indonesian') + ['though']
#stoplist

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,2))
# matrix of ngrams
ngrams = c_vec.fit_transform(dataSB['textdataClean'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_

df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})
df_ngram[0:40]

In [ ]:
#list(df_ngram('bigram/trigram'))

# save to exccel

In [ ]:
filedisimpan='data_bigram.xlsx'
df_ngram[0:100].to_excel(filedisimpan, index = False, header=True)